# Kevin Sun
## Using Twitter Sentiment Analysis to Predict Trends in Airline Brands
The goal of this open project is to use the classic, over-used twitter sentiment analysis in a more meaningful way. In ths past, I heard of friends using this analysis in predicting Ethereum prices, political elections, and much more. While scrolling through example training data, I found specific tweets toward airlines, and thought it would be cool to view the sentiments towards United Airlines, JetBlue, etc. over time. Hopefully my visualizations can detect rises and drops in the brand, for example a negative drop when United Airlines ejected the asian man a few months ago.

In [1]:
import tweepy
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

import csv
import json
import datetime as dt
import time
import os
import sys
import re

import pandas as pd
import nltk
from nltk.corpus import stopwords

### Data Collection: Scraping Tweets
First, let's scrape some tweets for our first keyword. For the scraping algorithm, I downloaded https://github.com/Jefferson-Henrique/GetOldTweets-python to account for the twitter limits and other important config stuff. Here is the command I used to generate the tweets with query "#unitedairlines" since the beginning of 2017, capping at 1,000,000 tweets:

In [2]:
# python Exporter.py --querysearch "#unitedairlines" --since 2017-01-01 --until 2017-10-31 --maxtweets 1000000
# Exporter.py is in the GetOldTweets-python folder

### Cleaning
Tweets contain a lot of noise. Before we start training the sentiment analyzer from the twitter training data, we first need to remove noise and preprocess tweets with the following steps:
<ol style="ordered">
<li>Convert the tweets to lower case.</li>
<li>Remove all URLs</li>
<li>Remove all twitter user referencing with "@username"</li>
<li>Remove the "#" from hashtags but keep the word</li>
<li>Punctuations and additional white spaces</li>
<li>As always, remove stopwords</li>
</ol>
<b>The scraped tweets will be used later to test the classifier.</b>

In [12]:
def preprocessTweet(tweet): # preprocess tweets
    tweet = re.sub(r'\d+', '', str(tweet)) # remove numbers
    tweet = tweet.lower() # convert to lower case
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet) # convert www.* or https?://* to URL
    tweet = re.sub('@[^\s]+','AT_USER',tweet) # convert @username to AT_USER
    tweet = re.sub('[\s]+', ' ', tweet) # remove additional white spaces
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet) # replace #word (hashtags) with word
    tweet = tweet.strip('\'"') # trim
    return tweet    

def replaceTwoOrMore(s):
    #look for 2 or more repetitions of character and replace with the character itself
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    return pattern.sub(r"\1\1", s)

preprocessTweet("@VirginAmerica @virginmedia I'm flying your #fabulous #Seductive skies again! U take all the #stress away from travel http://t.co/ahlXHhKiyn")

"AT_USER AT_USER i'm flying your fabulous seductive skies again! u take all the stress away from travel URL"

In [4]:
def getStopWordList(stopWordListFileName): # get stopword list
    sw = [] # create list of stopwords
    
    nltk_stopwords = stopwords.words('english')
    for w in nltk_stopwords:
        sw.append(w)
    sw.append('AT_USER') # special stopwords from preprocessTweet function
    sw.append('URL')
    
    fp = open(stopWordListFileName, 'r') # load in any more custom stopwords from file
    line = fp.readline()
    while line:
        word = line.strip()
        sw.append(word)
        line = fp.readline()
    fp.close()
    return sw


stopword_list = getStopWordList('data/stopwords.txt')
print('Stopword List: ', stopword_list)

Stopword List:  ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don

### Feature Extraction
An import step of any classifier is deciding what features are relevant. In the case of sentiment analysis, the presence or absence of words in a tweet would be the features.<br>
For example, if a tweet contained the word "terrible", it should likely be classified as "negative" sentiment.<br><br>
In the training data, we can split each tweet into words and add each word to the feature vector if it's a relevant indication of sentiment (we can filter out unimportant characters or words).<br><br>
Tf-idf might be able to extract features for me, but let's do it manually this time

In [5]:
def getFeatureVector(tweet): 
    featureVector = []
    #split tweet into words
    words = tweet.split()
    for w in words:
        w = replaceTwoOrMore(w) # replace two or more with two occurrences
        w = w.strip('\'"?,.') # strip punctuation
        val = re.search(r"^[a-zA-Z][a-zA-Z0-9]*$", w) # check if the word starts with an alphabet
        if(w in stopword_list or val is None): # ignore if it is a stop word
            continue
        else:
            featureVector.append(w.lower())
    return featureVector
 
getFeatureVector(preprocessTweet("@VirginAmerica @virginmedia I'm flying your #fabulous #Seductive skies again! U take all the #stress away from travel http://t.co/ahlXHhKiyn"))

['flying',
 'fabulous',
 'seductive',
 'skies',
 'u',
 'take',
 'stress',
 'away',
 'travel']

### Training Step: Naive Bayes Classifier
Now it's time to extract the word features from the actual training data downloaded from https://www.kaggle.com/crowdflower/twitter-airline-sentiment and https://www.crowdflower.com/data-for-everyone/. It contains twitter data scraped from February of 2015 for many airlines.<br><br>
Contributors were asked to first classify positive, negative, and neutral tweets, followed by categorizing negative reasons (such as “late flight” or “rude service”).

In [6]:
airline_train = pd.read_csv("data/train/Airline-Sentiment-2-w-AA.csv", encoding="ISO-8859-1") #latin1 encoding
airline_tweets = []
airline_featurelist = []
for i in range(len(airline_train)):
    sentiment = airline_train['airline_sentiment'][i]
    tweet = airline_train['text'][i]
    preprocessedTweet = preprocessTweet(tweet)
    featureVector = getFeatureVector(preprocessedTweet)
    airline_featurelist.extend(featureVector)
    airline_tweets.append((featureVector, sentiment))
        
def extract_features(tweet): # Determine if tweet contains a feature word
    tweet_words = set(tweet)
    features = {}
    for word in airline_featurelist:
        features['contains(%s)' % word] = (word in tweet_words)
    return features

airline_featurelist = list(set(airline_featurelist)) # remove airline_featurelist duplicates
airline_featurelist

['singapore',
 'equality',
 'registered',
 'gabriela',
 'announce',
 'solar',
 'fiousairways',
 'tracker',
 'flew',
 'restrm',
 'cheap',
 'nantucket',
 'remedy',
 'hero',
 'takeoff',
 'fcuk',
 'notgoodenough',
 'lane',
 'belabor',
 'middleeast',
 'popular',
 'thousandth',
 'higher',
 'hassles',
 'stock',
 'sec',
 'comps',
 'inconsistent',
 'ruining',
 'blank',
 'deleted',
 'grassy',
 'vacay',
 'contain',
 'kate',
 'lucia',
 'notch',
 'lastflightofthenight',
 'silly',
 'simple',
 'removes',
 'vcp',
 'nancy',
 'momma',
 'affiliate',
 'tasha',
 'instructions',
 'nyfw',
 'announcements',
 'kangaroos',
 'tea',
 'disregard',
 'terrific',
 'firm',
 'interfering',
 'inquiry',
 'bil',
 'smallest',
 'declared',
 'closepwcs',
 'medically',
 'intentionally',
 'refreshments',
 'und',
 'collection',
 'caribbean',
 'periolodicly',
 'newplanesmell',
 'carta',
 'amateurs',
 'number',
 'shaker',
 'dmv',
 'stating',
 'range',
 'enoughisenough',
 'strollers',
 'cousin',
 'gobankrupt',
 'considerably',
 'p

Use a Naive Bayes (NB) Classifier since every feature impacts the output label. The NBClassifier begins by calculating the prior probability of each label by checking frequency of each label in the training set.<br><br>
According to a Stanford University research (https://nlp.stanford.edu/pubs/sidaw12_simple_sentiment.pdf), Naive Bayes outperforms Support Vector Machines on short snippet sentiment classification tasks.<br><br>
Referenced: http://www.nltk.org/book/ch06.html

In [ ]:
airline_training_set = nltk.classify.util.apply_features(extract_features, airline_tweets)
airline_NBClassifier = nltk.NaiveBayesClassifier.train(airline_training_set)

Dump the model so that we can load it in at any time. <b>Note: Above time-consuming training steps can now be skipped!</b>

In [7]:
from sklearn.externals import joblib
# joblib.dump(airline_NBClassifier, 'data/airline_NBClassifier.pkl') 
airline_NBClassifier = joblib.load('data/airline_NBClassifier.pkl') 

### Model Evaluation
Now it's time to use our trained model on new testing data: the recent tweets we scraped in the beginning of the notebook! We will use the model to classify the sentiments of texts from <b>#unitedairlines</b>, <b>#jetblue</b>, <b>#virginamerica</b>, <b>#deltaairlines</b>, and <b>#southwestairlines</b>. But first, let's test it on a random tweet I found in the csv data files.

In [20]:
airline_NBClassifier.classify(extract_features(getFeatureVector(preprocessTweet("RT @lammdogg: More pics of the 787-9 delivery in Chicago yesterday @united @weareunited #unitedairlines #beingunited https://t.co/v0sO7odE0s"))))

'neutral'

In [8]:
unitedairlines_df = pd.read_csv('data/unitedairlines.csv', encoding="ISO-8859-1")
unitedairlines_df['sentiment'] = ''
unitedairlines_df.head()

,username,date,retweets,favorites,text,hashtags,sentiment
0,hallantongs,2017-10-30 19:04,0,0,#united #unitedairlines #gru #guarulhosairport...,#united #unitedairlines #gru #guarulhosairport...,
1,miguelfbono,2017-10-30 19:02,0,0,#App de #UnitedAirlines amplÃ­a su alcance - h...,#App #UnitedAirlines #viajes #aviacion,
2,kolekts,2017-10-30 18:17,0,0,Check out #Vintage #UNITEDAIRLINES - 6 #Teaspo...,#Vintage #UNITEDAIRLINES #Teaspoons #Oneida #A...,
3,MGFields22,2017-10-30 18:11,0,1,Pilot apologized for delay. Learned on taxiway...,#unitedAIRLINES,
4,JenoxCyde,2017-10-30 17:56,1,2,ANOTHER ONE poor!!!!!!!! #unitedairlines #hall...,#unitedairlines #halloweenCostume,


In [9]:
for i in range(len(unitedairlines_df)):
    tweet = unitedairlines_df['text'][i]
    unitedairlines_df.loc[i, 'sentiment'] =  airline_NBClassifier.classify(extract_features(getFeatureVector(preprocessTweet(tweet))))
unitedairlines_df.head()

,username,date,retweets,favorites,text,hashtags,sentiment
0,hallantongs,2017-10-30 19:04,0,0,#united #unitedairlines #gru #guarulhosairport...,#united #unitedairlines #gru #guarulhosairport...,negative
1,miguelfbono,2017-10-30 19:02,0,0,#App de #UnitedAirlines amplÃ­a su alcance - h...,#App #UnitedAirlines #viajes #aviacion,negative
2,kolekts,2017-10-30 18:17,0,0,Check out #Vintage #UNITEDAIRLINES - 6 #Teaspo...,#Vintage #UNITEDAIRLINES #Teaspoons #Oneida #A...,neutral
3,MGFields22,2017-10-30 18:11,0,1,Pilot apologized for delay. Learned on taxiway...,#unitedAIRLINES,negative
4,JenoxCyde,2017-10-30 17:56,1,2,ANOTHER ONE poor!!!!!!!! #unitedairlines #hall...,#unitedairlines #halloweenCostume,negative


In [10]:
unitedairlines_df.to_csv('data/unitedairlines_classified.csv', index=False)

For JetBlue, there was some problems with preprocessing, so I tweaked it to remove numbers before .lower() and make sure everything is in str format.

In [13]:
jetblue_df = pd.read_csv('data/jetblue.csv', encoding="ISO-8859-1") # latin1 encoding
jetblue_df['sentiment'] = ''
jetblue_df.head()
for i in range(len(jetblue_df)):
    tweet = jetblue_df['text'][i]
    jetblue_df.loc[i, 'sentiment'] =  airline_NBClassifier.classify(extract_features(getFeatureVector(preprocessTweet(tweet))))
    print(i, ':', tweet)
jetblue_df.to_csv('data/jetblue_classified.csv', index=False)
jetblue_df.head()

0 : I spent hundreds of dollars flying @JetBlue #flight603 cross county to have the ability to watch TV. This is what I get. #jetblue #tv #espn pic.twitter.com/KATQe3Ga9F
1 : There are a lot of reasons to like #JetBlue
2 : The verdict is in, #JetBlue you are officially the best airline and terminal in my books! TV's, wifi, iPads, kids play area #winning
3 : Thanks SFO. Now I am getting in at midnight. Makes me too late to catch Bart train. #JetBlue
4 : History in #Boston Stuck in #Boston waiting on #JetBlue flight home.. #JetBlue https://www. instagram.com/p/Ba4yQvVAjHT/
5 : @JetBlue #JetBlue #Cali
6 : @AmericanAir @JetBlue can someone in customer svc PLEASE help these people find their luggage! #AmericanAirlines #JetBlue #cstomerservice
7 : #JetBlue compte ouvrir une 4e ligne vers #HaÌøti #Orlando http:// airinfo.org/2017/04/12/jet blue-compte-ouvrir-une-4e-ligne-vers-haiti/ Û_ via @AirInfoAviation
8 : This looks fun. I bet it sucks in winter. Stuck in #Boston for awhile thanks to #j

68 : My screen didnt work. But even with that, @JetBlue still the best airline for domestic flights #JetBlue pic.twitter.com/bFEjIzbRC8
69 : Nothing like a sunrise from the air. Thanks #jetBlue for the ride. pic.twitter.com/vzW2Iz5x6b
70 : @JetBlue #NYC Here I Come. #jetblue Can I get a retweet ? #travel #miami #saturday pic.twitter.com/lhOZuI7Fjt
71 : Good morning. #view . #morning #sunrise #flying #jetblue #travel #vacation #vacationmode https://www. instagram.com/p/BayfkArF8KC/
72 : Goodbye Tampa. #jetblue . #tampa #travel #vacation #vacationmode @Florida https://www. instagram.com/p/BayaFiplDQ2/
73 : Booked my first @JetBlue flight @LBAirport #JetBlue feels wrong not flying @AlaskaAir and earning more miles. I couldnÛªt beat the price
74 : Thank you #jetblue for Tv. Watching World Series in flight!!
75 : #Boeing #JetBlue #BMW PR Firm initiates the Global connection of the newest designs' launch of communication firm BridgeStone Media Group
76 : Boeing #JetBlue #BMW PR Firm initiat

137 : Watching the world series at 30000 ft pretty cool jet blue oh and they give u free wi fi to boot #jetblue ... http:// fb.me/3cjWtcpFf
138 : #joy #flight462 #jetblue #jetblue462 from #sju #pr to #loganinternationalairport got my abuelita from... http:// fb.me/4aB5plUNM
139 : #joy #flight462 #jetblue #jetblue462 from #sju #pr to #loganinternationalairport got myÛ_ https://www. instagram.com/p/BaseLVmhPyi/
140 : #Portugal #JetBlue #Trending #Travel http:// bit.ly/2gEsXAj pic.twitter.com/SO4pdK8frY
141 : Shoutout to #JetBlue employees in PBI for returning my phone left at Gate 16 last week! Many thanks #PBIA workers for help looking for it!
142 : #Portugal #JetBlue #Trending #Travel http:// bit.ly/2zDyD5e pic.twitter.com/TZO5RVGzIT
143 : #Portugal #JetBlue #Trending #Travel http:// bit.ly/2zGcsLz
144 : #Portugal #JetBlue #Trending #Travel http:// bit.ly/2xmOlkn pic.twitter.com/rJbTs2WMM9
145 : #Portugal #JetBlue #Trending #Travel http:// bit.ly/2h9sF5d pic.twitter.com/6bJgEZXZZK
146

207 : . @JetBlue expands into the bus business. #jetbluebus #jetblue (cc: @LaurieAMeacham @MHJohnston ) https:// twitter.com/i/moments/9229 99191403610113 Û_
208 : @JetBlue flight 490 I paid for a 2 hour airplane ride to JFK. now I'm stuck in VA n your solution is a 6 hour bus ride to JFK. #jetblue
209 : Vuelo NYC (JFK ) - Fort Laiderdale por Delta ida y vuelta $268 por #Delta o por #JetBlue
210 : Broken Toilets are Keeping an Airline From Adding Seats to Their Planes - View from the Wing #JetBlue ... http:// aviation.trendolizer.com/2017/10/broken -toilets-are-keeping-an-airline-from-adding-seats-to-their-planes---view-from-the-wing.html Û_ pic.twitter.com/Nl8gQIY6Fm
211 : Doodling a sassy airline robot on my plane #JetBlue pic.twitter.com/Fod9Q62Njx
212 : Warm days in both North and South. Ok not warm. Hot. #upintheair #jetblue #onlocation Û_ https://www. instagram.com/p/BapipdYghDE/
213 : Delayed #travelproblems #delayed #delayedflight #pbi to #bdl #instatravel #jetblue #flight 

269 : JetBlue Airways Corporation http:// cheapflightsforjetblue.com/jetblue-com/ #jetblue
270 : Jetblue Flights http:// cheapflightsforjetblue.com/jetblue-flight s/ Û_ #jetblue
271 : Jetblue Cheap Tickets http:// cheapflightsforjetblue.com/jetblue-cheap- tickets/ Û_ #jetblue
272 : Jetblue Cheap Flights http:// cheapflightsforjetblue.com/jetblue-cheap- flights/ Û_ #jetblue
273 : Jetblue Cheap Fare http:// cheapflightsforjetblue.com/jetblue-cheap- fare/ Û_ #jetblue
274 : Jetblue http:// cheapflightsforjetblue.com/jetblue/ #jetblue
275 : Cheap Flights For Jetblue http:// cheapflightsforjetblue.com / #jetblue
276 : JetBlue just gets me @JetBlue #Astrology #JetBlue #Jetstrology pic.twitter.com/f4i3QxdShP
277 : Stuck on airplane delayed take off 1h42m and counting because of "paperwork technical issue". Really #JetBlue ? Someone screwed up big time.
278 : JetBlue backs air travel gifting app Skyhour - PE Hub: https://www. pehub.com/2017/10/jetblu e-backs-air-travel-gifting-app-skyhour/#

...

13407 : #JetBlue always comes through
13408 : Never flying #southwest again. Forever flying #JetBlue
13409 : Heading out back to NYC! #SFO #NY #NYC #jetblue #2017 #graditude #love #newyear #howdoplanesfly https://www. instagram.com/p/BO5ADSYBUlQ/
13410 : California here I come! #California #holidays #Coming -home #family #blessed #nyc #jetblue Û_ https://www. instagram.com/p/BO5ADJth-eB/
13411 : 2nd stop tonight Come party with me and my #JetBlue family @mannyvaldez (89-25 130thÛ_ https://www. instagram.com/p/BO4-9x_gPGO/
13412 : #TravelNews #JetBlue launches new daily non-stop flights from Fort Lauderdale to #Aruba ! #Travel #Vacation http:// bit.ly/2ieACEQ
13413 : #travel #ttot #traveldeal #Grenada #airfare #JetBlue https:// twitter.com/SecretFlying/s tatus/817009383590739968 Û_
13414 : IÛªm flying from BDL to SJU Jan 7 aka 1 day!!!!! So excited!!! #JetBlue
13415 : Traveling Plans? #JetBlue Has Flights For $34 Right Now! http:// goo.gl/UDoK8f pic.twitter.com/FVXYP8c4bv
13416 : #J

13470 : For at least the next few years, #JetBlue plans to keep the Embraer fleet size unchanged.
13471 : VR headsets being passed out before we even get to Vegas #jetblue #ces2017
13472 : JetBlue Announces Flash Winter Fare Sale http:// tinyurl.com/hskpwpk #jetblue #flashsale #ffmiles #thebigwintersale
13473 : But Truck Day in only 33... #BeepBeep #TheFort #JetBlue https:// twitter.com/redsox/status/ 816645413495250944 Û_
13474 : #jetblue really? Delayed departure now sitting waiting for a gate for 20 minutes? Oy!! #272 @jetblue
13475 : Inaugural flight from #FtLauderdale to #Aruba ! #jetblue Pictures taken by @ARUBA_AIRPORT pic.twitter.com/otlOkKHBSk
13476 : Urgh Having problems to use my points to buy a ticket @JetBlue #Jetblue
13477 : Back on my daily grind, have a great day everyone! #vacationsover #jetblue #techops pic.twitter.com/rGW3vrLwwY
13478 : FLL to San Juan Puerto Rico $105 RT nonstop on #JetBlue travel departing Jan-March, multiple dates #suzzstravels
13479 : #JetBlue s

13534 : The @Verge and #JetBlue are turning flights to #CES into #VR getaways http://www. theverge.com/ces/2017/1/3/1 4145492/verge-cardboard-vr-360-video-jetblue-ces-2017 Û_
13535 : The #Verge and #JetBlue are turning flights to #CES into #VR getaways http://www. theverge.com/ces/2017/1/3/1 4145492/verge-cardboard-vr-360-video-jetblue-ces-2017?utm_campaign=theverge&utm_content=entry&utm_medium=social&utm_source=twitter Û_ via @Verge
13536 : Well it certainly wasn't a coordinated publicity stunt. They'd never do something like that! @IvankaTrump #JetBlue
13537 : Still #traumatized from my @JetBlue flight last night after I was #discriminated against. Waiting email response. #jetblue #injustice
13538 : I saw a Jet Blue ticket counter at the airport, and now I can't stop Singing "What You Want" #LegallyBlonde #JetBlue #ThankYou #WhatYouWant
13539 : T5 is bumpin' this morning! Loving the Backstreet Boys playlist @JetBlue #letsdance #JetBlue
13540 : @JetBlue has the most rude workers and

13598 : @JetBlue JFK wheelchair asst a disaster this AM! We are exhausted from this experience .What happened 2U #JetBlue
13599 : #Jet #JetBlue #Airbus #A320 #A320Series #AirbusLovers #AirbusFans #FortLauderdale #FLL #KFLL Û_ https://www. instagram.com/p/BOsku3OB67L/
13600 : #Jet #JetBlue #Airbus #A320 #A320Series #AirbusLovers #AirbusFans #FortLauderdale #FLL #KFLL Û_ https://www. instagram.com/p/BOr8Bf5hsnd/
13601 : Remind me never to fly #jetBlue ever ever again.
13602 : 6 #USA #airlines #licensed to #offer #flights to #Cuba #AmericanAirlines #business #JetBlue http:// arzillion.com/S/EZSzTi pic.twitter.com/1o72yRfzku
13603 : nan
13604 : nan
13605 : nan
13606 : nan
13607 : nan
13608 : nan


,username,date,retweets,favorites,text,hashtags,sentiment
0,MikeZappa,10/30/17 19:45,0.0,1.0,I spent hundreds of dollars flying @JetBlue #f...,#flight603 #jetblue #tv #espn,negative
1,smsbda,10/30/17 18:06,0.0,0.0,There are a lot of reasons to like #JetBlue,#JetBlue,negative
2,BdaMichelle,10/30/17 18:00,1.0,16.0,"The verdict is in, #JetBlue you are officially...",#JetBlue #winning,positive
3,samtweets33,10/30/17 17:54,0.0,0.0,Thanks SFO. Now I am getting in at midnight. M...,#JetBlue,negative
4,wmjackson,10/30/17 17:29,0.0,0.0,History in #Boston Stuck in #Boston waiting on...,#Boston #Boston #JetBlue #JetBlue,negative


In [14]:
virginamerica_df = pd.read_csv('data/virginamerica.csv', encoding="ISO-8859-1")
virginamerica_df['sentiment'] = ''
virginamerica_df.head()
for i in range(len(virginamerica_df)):
    tweet = virginamerica_df['text'][i]
    virginamerica_df.loc[i, 'sentiment'] =  airline_NBClassifier.classify(extract_features(getFeatureVector(preprocessTweet(tweet))))
virginamerica_df.to_csv('data/virginamerica_classified.csv', index=False)
virginamerica_df.head()

,username,date,retweets,favorites,text,hashtags,sentiment
0,AbbyReyes114,2017-10-30 18:35,0,0,"Bruce Weight ft Xannyboi ""BUENO"" (OFFICIAL VID...",#bueno #virginamerica #MondayMotivation #Music...,neutral
1,kerrykeith,2017-10-30 09:05,0,0,#VX #virginamerica #n362va #vx362 #virgin #ane...,#VX #virginamerica #n362va #vx362 #virgin #ane...,positive
2,goldmind,2017-10-30 06:41,0,0,Welcome to the ânewâ post-merger #virginam...,#virginamerica #alaskaairline,positive
3,KarlHab,2017-10-29 10:28,2,13,Shot this years ago from a little R22 #virgina...,#virginamerica #airbus #losangeles #lax #Aviation,positive
4,mybanktracker,2017-10-28 17:51,0,0,The Best Options to Replace Virgin America Cre...,#creditcard #virginamerica #airlines,neutral


In [15]:
deltaairlines_df = pd.read_csv('data/deltaairlines.csv', encoding="ISO-8859-1")
deltaairlines_df['sentiment'] = ''
deltaairlines_df.head()
for i in range(len(deltaairlines_df)):
    tweet = deltaairlines_df['text'][i]
    deltaairlines_df.loc[i, 'sentiment'] =  airline_NBClassifier.classify(extract_features(getFeatureVector(preprocessTweet(tweet))))
deltaairlines_df.to_csv('data/deltaairlines_classified.csv', index=False)
deltaairlines_df.head()

,username,date,retweets,favorites,text,hashtags,sentiment
0,miles_report,10/30/17 19:36,0.0,0.0,Plane carrying Oklahoma City Thunder likely da...,#DeltaAirLines #t14909,negative
1,sennawcf1,10/30/17 18:56,0.0,1.0,Congratulations to #deltaairlines trip yyz and...,#deltaairlines,negative
2,coastalbri,10/30/17 18:42,0.0,0.0,Have to say awesome customer service from #del...,#deltaairlines,negative
3,gustlfm,10/30/17 18:01,0.0,0.0,Long Island Sound looking pretty! #deltaairlin...,#deltaairlines,positive
4,RetroSwank1,10/30/17 16:52,0.0,0.0,Keep up the good work #DeltaAirlines ! On a fl...,#DeltaAirlines,positive


In [16]:
southwestairlines_df = pd.read_csv('data/southwestairlines.csv', encoding="ISO-8859-1")
southwestairlines_df['sentiment'] = ''
southwestairlines_df.head()
for i in range(len(southwestairlines_df)):
    tweet = southwestairlines_df['text'][i]
    southwestairlines_df.loc[i, 'sentiment'] =  airline_NBClassifier.classify(extract_features(getFeatureVector(preprocessTweet(tweet))))
southwestairlines_df.to_csv('data/southwestairlines_classified.csv', index=False)
southwestairlines_df.head()

,username,date,retweets,favorites,text,hashtags,sentiment
0,LisaBPet68,2017-10-30 19:46,0,0,Innovation #Intern - #Summer 2018 at #Southwes...,#Intern #Summer #SouthwestAirlines #Dallas #job,neutral
1,shariatswa,2017-10-30 19:45,0,0,Check out this #job : Senior #System #Engineer...,#job #System #Engineer #SouthwestAirlines #Dallas,negative
2,bevbrent1,2017-10-30 19:10,0,0,#SouthwestAirlines : #Technical Writing - Cent...,#SouthwestAirlines #Technical #Intern #Dallas ...,negative
3,jenellr2,2017-10-30 19:07,0,0,#SouthwestAirlines is looking for a Ramp #Agen...,#SouthwestAirlines #Agent #Milwaukee #job,positive
4,sebradshaw,2017-10-30 19:05,0,0,#SouthwestAirlines : Political #Communication ...,#SouthwestAirlines #Communication #Intern #Dal...,negative


### Data Visualization
The first data visualization will be a histogram-type graph. The x-axis will have time ranges for every day in 2017 (so far). The y-axis will be the number of tweets. Within each time range, I will draw rectangles for the number of negative, neutral, and positive tweets.<br>
To accomplish this, let's prepare the data so that d3 can read it in easier.

In [90]:
# since we are going to do repititive actions for all 5 airlines, let's create these useful dictionary
d = {'unitedairlines': unitedairlines_df, 'jetblue': jetblue_df, 'virginamerica': virginamerica_df, 
       'deltaairlines': deltaairlines_df, 'southwestairlines': southwestairlines_df}

In [91]:
# Checkpoint to reset dataframes from above
for key in d:
    d[key] = pd.read_csv('data/' + key + '_classified.csv')

Taking a look at the 'date' columns, some of the strings are in different formats, so let's get them all in the format of '%Y-%m-%d'. Also, let's drop and NaN rows.

In [92]:
import datetime

for key in d:
    for i in range(len(d[key])):
        if key in ['jetblue', 'deltaairlines']: # these date format is different from rest
            try:
                d[key].loc[i, 'date'] = str(datetime.datetime.strptime(str(d[key]['date'][i]).split(" ")[0], '%m/%d/%y').strftime('%Y-%m-%d'))
            except:
                print('error:', d[key]['date'][i])
        else:
            d[key].loc[i, 'date'] = str(d[key]['date'][i]).split(" ")[0] # remove time from date
    d[key] = d[key].dropna(how='any') # drop rows with NaN values
    print("Done with", key)
d['jetblue']

Done with unitedairlines
error: nan
error: nan
error: nan
error: nan
error: nan
error: nan
Done with jetblue
Done with virginamerica
error: nan
Done with deltaairlines
Done with southwestairlines


,username,date,retweets,favorites,text,hashtags,sentiment
0,MikeZappa,2017-10-30,0.0,1.0,I spent hundreds of dollars flying @JetBlue #f...,#flight603 #jetblue #tv #espn,negative
1,smsbda,2017-10-30,0.0,0.0,There are a lot of reasons to like #JetBlue,#JetBlue,negative
2,BdaMichelle,2017-10-30,1.0,16.0,"The verdict is in, #JetBlue you are officially...",#JetBlue #winning,positive
3,samtweets33,2017-10-30,0.0,0.0,Thanks SFO. Now I am getting in at midnight. M...,#JetBlue,negative
4,wmjackson,2017-10-30,0.0,0.0,History in #Boston Stuck in #Boston waiting on...,#Boston #Boston #JetBlue #JetBlue,negative
5,PotroRob,2017-10-30,0.0,0.0,@JetBlue #JetBlue #Cali,#JetBlue #Cali,positive
6,Pamelastonenyc,2017-10-30,0.0,0.0,@AmericanAir @JetBlue can someone in customer ...,#AmericanAirlines #JetBlue #cstomerservice,negative
7,jonaslaurince,2017-10-30,2.0,2.0,#JetBlue compte ouvrir une 4e ligne vers #HaÌø...,#JetBlue #Ha #Orlando,neutral
8,wmjackson,2017-10-30,0.0,0.0,This looks fun. I bet it sucks in winter. Stuc...,#Boston #jetblue,negative
9,DouglasCopley,2017-10-30,0.0,0.0,Julie Porro from #JetBlue advocating for bette...,#JetBlue #CyberSecurity,positive


Now it's time to populate the final csv! This airline_data.csv will contain data for all 5 airlines, with it's days and the sentiment counts for each day.

In [103]:
airline_data = pd.DataFrame(columns=['airline', 'day', 'negative_count', 'neutral_count', 'positive_count'])

def addDataFromAirline(airline):
    global d # global variables
    global airline_data
    checked_days = [] # keep track of which days we have already added to airline_data
    for i in range(len(d[airline])):
        cur_date = d[airline]['date'][i]
        if cur_date not in checked_days:
            checked_days.append(cur_date)
            # temp is a temporary dataframe with all rows on the current date
            temp = d[airline][pd.to_datetime(d[airline]["date"]).isin(pd.date_range(cur_date, cur_date))]
            temp = temp.reset_index()
            neg = 0 # negative count
            neu = 0 # neutral count
            pos = 0 # positive count
            for j in range(len(temp)):
                if temp['sentiment'][j] == 'negative':
                    neg += 1
                elif temp['sentiment'][j] == 'positive':
                    pos += 1
                else:
                    neu += 1
            temp_df = pd.DataFrame([[airline, cur_date, neg, neu, pos]], columns=['airline', 'day', 'negative_count', 'neutral_count', 'positive_count'])
            print(temp_df) # keep track of progress
            airline_data = airline_data.append(temp_df, ignore_index=True)
        
for key in d:
    addDataFromAirline(key) # add data from all airlines in the dictionary: d
        
airline_data

          airline         day  negative_count  neutral_count  positive_count
0  unitedairlines  2017-10-30              33              6               4
          airline         day  negative_count  neutral_count  positive_count
0  unitedairlines  2017-10-29              44             11               8
          airline         day  negative_count  neutral_count  positive_count
0  unitedairlines  2017-10-28              14              4               8
          airline         day  negative_count  neutral_count  positive_count
0  unitedairlines  2017-10-27              33             13              12
          airline         day  negative_count  neutral_count  positive_count
0  unitedairlines  2017-10-26              32             19              11
          airline         day  negative_count  neutral_count  positive_count
0  unitedairlines  2017-10-25              36              9              11
          airline         day  negative_count  neutral_count  positive_count

          airline         day  negative_count  neutral_count  positive_count
0  unitedairlines  2017-09-04              45             15               9
          airline         day  negative_count  neutral_count  positive_count
0  unitedairlines  2017-09-03              21              5               6
          airline         day  negative_count  neutral_count  positive_count
0  unitedairlines  2017-09-02              22             13               8
          airline         day  negative_count  neutral_count  positive_count
0  unitedairlines  2017-09-01              38             20               7
          airline         day  negative_count  neutral_count  positive_count
0  unitedairlines  2017-08-31              52             10               6
          airline         day  negative_count  neutral_count  positive_count
0  unitedairlines  2017-08-30              46             19              12
          airline         day  negative_count  neutral_count  positive_count

          airline         day  negative_count  neutral_count  positive_count
0  unitedairlines  2017-07-04              60             13              11
          airline         day  negative_count  neutral_count  positive_count
0  unitedairlines  2017-07-03              51              7               5
          airline         day  negative_count  neutral_count  positive_count
0  unitedairlines  2017-07-02              56             15              18
          airline         day  negative_count  neutral_count  positive_count
0  unitedairlines  2017-07-01              78             17               7
          airline         day  negative_count  neutral_count  positive_count
0  unitedairlines  2017-06-30             133            144              35
          airline         day  negative_count  neutral_count  positive_count
0  unitedairlines  2017-06-29              70             13              12
          airline         day  negative_count  neutral_count  positive_count

   airline         day  negative_count  neutral_count  positive_count
0  jetblue  2017-10-03              17              9               9
   airline         day  negative_count  neutral_count  positive_count
0  jetblue  2017-10-02              15              6               9
   airline         day  negative_count  neutral_count  positive_count
0  jetblue  2017-10-01               7              9              11
   airline         day  negative_count  neutral_count  positive_count
0  jetblue  2017-09-30               9             11              23
   airline         day  negative_count  neutral_count  positive_count
0  jetblue  2017-09-29              22             12              20
   airline         day  negative_count  neutral_count  positive_count
0  jetblue  2017-09-28              30             19              27
   airline         day  negative_count  neutral_count  positive_count
0  jetblue  2017-09-27              21             10              23
   airline         d

   airline         day  negative_count  neutral_count  positive_count
0  jetblue  2017-07-17              33             13              16
   airline         day  negative_count  neutral_count  positive_count
0  jetblue  2017-07-16              22             11              17
   airline         day  negative_count  neutral_count  positive_count
0  jetblue  2017-07-15              20             11              17
   airline         day  negative_count  neutral_count  positive_count
0  jetblue  2017-07-14              30             10              12
   airline         day  negative_count  neutral_count  positive_count
0  jetblue  2017-07-13              48              5               7
   airline         day  negative_count  neutral_count  positive_count
0  jetblue  2017-07-12              33              4              19
   airline         day  negative_count  neutral_count  positive_count
0  jetblue  2017-07-11              25              9              13
   airline         d

...

0              35  
             airline         day  negative_count  neutral_count  \
0  southwestairlines  2017-09-14               7              2   

   positive_count  
0              45  
             airline         day  negative_count  neutral_count  \
0  southwestairlines  2017-09-13               4              5   

   positive_count  
0              60  
             airline         day  negative_count  neutral_count  \
0  southwestairlines  2017-09-12               5              2   

   positive_count  
0              36  
             airline         day  negative_count  neutral_count  \
0  southwestairlines  2017-09-11               5              5   

   positive_count  
0              34  
             airline         day  negative_count  neutral_count  \
0  southwestairlines  2017-09-10               3              2   

   positive_count  
0              33  
             airline         day  negative_count  neutral_count  \
0  southwestairlines  2017-09-09      

             airline         day  negative_count  neutral_count  \
0  southwestairlines  2017-07-16               6              3   

   positive_count  
0              30  
             airline         day  negative_count  neutral_count  \
0  southwestairlines  2017-07-15               9              4   

   positive_count  
0              35  
             airline         day  negative_count  neutral_count  \
0  southwestairlines  2017-07-14              13              1   

   positive_count  
0              50  
             airline         day  negative_count  neutral_count  \
0  southwestairlines  2017-07-13              12              5   

   positive_count  
0              40  
             airline         day  negative_count  neutral_count  \
0  southwestairlines  2017-07-12               9              3   

   positive_count  
0              38  
             airline         day  negative_count  neutral_count  \
0  southwestairlines  2017-07-11              13          

0              25  
             airline         day  negative_count  neutral_count  \
0  southwestairlines  2017-05-17               5              3   

   positive_count  
0              36  
             airline         day  negative_count  neutral_count  \
0  southwestairlines  2017-05-16               4              1   

   positive_count  
0              24  
             airline         day  negative_count  neutral_count  \
0  southwestairlines  2017-05-15              12              1   

   positive_count  
0              25  
             airline         day  negative_count  neutral_count  \
0  southwestairlines  2017-05-14               9              2   

   positive_count  
0              28  
             airline         day  negative_count  neutral_count  \
0  southwestairlines  2017-05-13               7              0   

   positive_count  
0              24  
             airline         day  negative_count  neutral_count  \
0  southwestairlines  2017-05-12      

             airline         day  negative_count  neutral_count  \
0  southwestairlines  2017-03-23               6              4   

   positive_count  
0              15  
             airline         day  negative_count  neutral_count  \
0  southwestairlines  2017-03-22               3              0   

   positive_count  
0              21  
             airline         day  negative_count  neutral_count  \
0  southwestairlines  2017-03-21               5              4   

   positive_count  
0              15  
             airline         day  negative_count  neutral_count  \
0  southwestairlines  2017-03-20               2              1   

   positive_count  
0              14  
             airline         day  negative_count  neutral_count  \
0  southwestairlines  2017-03-19               2              4   

   positive_count  
0              17  
             airline         day  negative_count  neutral_count  \
0  southwestairlines  2017-03-18               3          

,airline,day,negative_count,neutral_count,positive_count
0,unitedairlines,2017-10-30,33,6,4
1,unitedairlines,2017-10-29,44,11,8
2,unitedairlines,2017-10-28,14,4,8
3,unitedairlines,2017-10-27,33,13,12
4,unitedairlines,2017-10-26,32,19,11
5,unitedairlines,2017-10-25,36,9,11
6,unitedairlines,2017-10-24,51,18,8
7,unitedairlines,2017-10-23,63,18,17
8,unitedairlines,2017-10-22,33,11,3
9,unitedairlines,2017-10-21,42,5,10


In [104]:
airline_data.to_csv('data/airline_data.csv', index=False)